# 02 - Preprocesamiento de Datos
## Kaggle Playground Series S5E6: Predicting Optimal Fertilizers

**Autor:** Félix  
**Fecha:** 3 de junio de 2025  
**Objetivo:** Preparar datos para modelos TIER 1 (Random Forest, LightGBM, XGBoost)

### 📋 Plan de Trabajo:
1. **Recrear variables del EDA** - Todas las features identificadas como importantes
2. **Encoding** - Preparar variables categóricas para los modelos
3. **Escalado** - Normalizar variables numéricas 
4. **División** - Train/Validation 80/20 con random_state=513
5. **Guardado** - Datasets listos para modelado en `/data/processed/`

## 📚 Importar Librerías Necesarias

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

# Configuraciones
warnings.filterwarnings('ignore')
np.random.seed(513)

# Crear directorio para datos procesados si no existe
processed_dir = '../data/processed'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
    print(f"✅ Directorio creado: {processed_dir}")
else:
    print(f"✅ Directorio ya existe: {processed_dir}")

print("Librerías importadas correctamente")

✅ Directorio ya existe: ../data/processed
Librerías importadas correctamente


## 📊 Cargar Datos de Entrenamiento y Prueba

In [2]:
# Cargar datasets originales
train_df = pd.read_csv('../data/train.csv').set_index('id')  # ← Establecer 'id' como índice
test_df = pd.read_csv('../data/test.csv').set_index('id')   # ← Establecer 'id' como índice
sample_submission = pd.read_csv('../data/sample_submission.csv')

print("=== INFORMACIÓN DE DATASETS ===\n")
print(f"📈 Train shape: {train_df.shape}")
print(f"📊 Test shape: {test_df.shape}")
print(f"📝 Sample submission shape: {sample_submission.shape}")

print("\n=== COLUMNAS DISPONIBLES ===\n")
print(f"Train columns: {list(train_df.columns)}")
print(f"Test columns: {list(test_df.columns)}")

print("\n=== VARIABLE OBJETIVO ===\n")
print(f"Tipos únicos de fertilizantes: {train_df['Fertilizer Name'].nunique()}")
print(f"Distribución target (top 5):")
print(train_df['Fertilizer Name'].value_counts().head())

# Verificar consistencia entre train y test
print("\n=== VERIFICACIÓN DE CONSISTENCIA ===\n")
train_cols = set(train_df.columns) - {'Fertilizer Name'}
test_cols = set(test_df.columns)  # test ya no tiene 'id' como columna (es el índice)

if train_cols == test_cols:
    print("✅ Train y test tienen las mismas features")
else:
    print("⚠️ Diferencias en columnas:")
    print(f"  Solo en train: {train_cols - test_cols}")
    print(f"  Solo en test: {test_cols - train_cols}")

print("\n=== PREVIEW DE DATOS ===\n")
print("Train (con índice 'id'):")
display(train_df.head(3))
print("\nTest (con índice 'id'):")
display(test_df.head(3))

=== INFORMACIÓN DE DATASETS ===

📈 Train shape: (750000, 9)
📊 Test shape: (250000, 8)
📝 Sample submission shape: (250000, 2)

=== COLUMNAS DISPONIBLES ===

Train columns: ['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name']
Test columns: ['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous']

=== VARIABLE OBJETIVO ===

Tipos únicos de fertilizantes: 7
Distribución target (top 5):
Fertilizer Name
14-35-14    114436
10-26-26    113887
17-17-17    112453
28-28       111158
20-20       110889
Name: count, dtype: int64

=== VERIFICACIÓN DE CONSISTENCIA ===

✅ Train y test tienen las mismas features

=== PREVIEW DE DATOS ===

Train (con índice 'id'):


,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
id,,,,,,,,,
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,27,69,65,Sandy,Millets,30,6,18,28-28
2,29,63,32,Sandy,Millets,24,12,16,17-17-17



Test (con índice 'id'):


,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
id,,,,,,,,
750000,31,70,52,Sandy,Wheat,34,11,24
750001,27,62,45,Red,Sugarcane,30,14,15
750002,28,72,28,Clayey,Ground Nuts,14,15,4


## ⚙️ Recrear Variables del EDA

### 🎯 Feature Engineering Estratégico
Recrearemos todas las variables importantes identificadas en el EDA:
- **Ratios NPK** - Relaciones entre nutrientes principales
- **Índices ambientales** - Combinaciones de variables climáticas
- **Categorización inteligente** - Niveles de nutrientes y condiciones
- **Interacciones** - Combinaciones suelo-cultivo
- **Features de balance** - Métricas de equilibrio nutricional

In [3]:
def apply_feature_engineering(df, is_train=True):
    """
    Aplica el mismo feature engineering del EDA a cualquier dataset
    """
    df_features = df.copy()
    
    print(f"🔧 Aplicando feature engineering a {'TRAIN' if is_train else 'TEST'}...")
    
    # 1. RATIOS DE NUTRIENTES (basado en alto MI de NPK)
    print("  1️⃣ Creando ratios de nutrientes...")
    df_features["N_P_ratio"] = df_features["Nitrogen"] / (df_features["Phosphorous"] + 1)
    df_features["N_K_ratio"] = df_features["Nitrogen"] / (df_features["Potassium"] + 1)
    df_features["P_K_ratio"] = df_features["Phosphorous"] / (df_features["Potassium"] + 1)
    df_features["Total_NPK"] = (
        df_features["Nitrogen"] + df_features["Phosphorous"] + df_features["Potassium"]
    )
    
    # 2. ÍNDICES AMBIENTALES
    print("  2️⃣ Creando índices ambientales...")
    df_features["Temp_Hum_index"] = (
        df_features["Temparature"] * df_features["Humidity"] / 100
    )
    df_features["Moist_Balance"] = df_features["Moisture"] - df_features["Humidity"]
    df_features["Environ_Stress"] = abs(df_features["Temparature"] - 25) + abs(
        df_features["Humidity"] - 65
    )
    
    # 3. CATEGORIZACIÓN INTELIGENTE
    print("  3️⃣ Creando categorías basadas en distribuciones...")
    df_features["Temp_Cat"] = pd.cut(
        df_features["Temparature"],
        bins=[0, 25, 30, 35, 100],
        labels=["Frío", "Templado", "Cálido", "Muy_Cálido"],
    )
    df_features["Hum_Cat"] = pd.cut(
        df_features["Humidity"],
        bins=[0, 50, 65, 80, 100],
        labels=["Muy_Baja", "Baja", "Media", "Alta"],
    )
    df_features["N_Level"] = pd.cut(
        df_features["Nitrogen"], bins=[0, 15, 25,100], include_lowest=True,  labels=["Bajo", "Medio", "Alto"]
    )
    df_features["K_Level"] = pd.cut(
        df_features["Potassium"], bins=[0, 4, 14, 100],  include_lowest=True, labels=["Bajo", "Medio", "Alto"]
    )
    df_features["P_Level"] = pd.cut(
        df_features["Phosphorous"], bins=[0, 10, 32,100], include_lowest=True,  labels=["Bajo", "Medio", "Alto"]
    )
    
    # 4. FEATURES DE INTERACCIÓN
    print("  4️⃣ Creando interacciones estratégicas...")
    df_features["Soil_Crop_Combo"] = (
        df_features["Soil Type"] + "_" + df_features["Crop Type"]
    )
    
    # 5. FEATURES DE BALANCE Y RATIOS AVANZADOS
    print("  5️⃣ Features de balance y ratios avanzados...")
    
    # Balance de macronutrientes
    df_features['NPK_Balance'] = abs(df_features['Nitrogen'] - df_features['Phosphorous']) + \
                                abs(df_features['Nitrogen'] - df_features['Potassium']) + \
                                abs(df_features['Phosphorous'] - df_features['Potassium'])
    
    # Índice de nutriente dominante
    max_nutrient = df_features[['Nitrogen', 'Phosphorous', 'Potassium']].max(axis=1)
    df_features['Dominant_NPK_Level'] = max_nutrient
    
    # Condiciones ambientales combinadas
    df_features['Temp_Moist_inter'] = df_features['Temparature'] * df_features['Moisture'] / 100
    
    # Listar nuevas features creadas
    new_features = [
        "N_P_ratio", "N_K_ratio", "P_K_ratio", "Total_NPK",
        "Temp_Hum_index", "Moist_Balance", "Environ_Stress",
        "Temp_Cat", "Hum_Cat", "N_Level", "K_Level", "P_Level",
        "Soil_Crop_Combo", 'NPK_Balance', 'Dominant_NPK_Level', 'Temp_Moist_inter'
    ]
    
    print(f"  ✅ {len(new_features)} nuevas features creadas")
    
    return df_features, new_features

# Aplicar feature engineering a ambos datasets
train_processed, new_features_list = apply_feature_engineering(train_df, is_train=True)
test_processed, _ = apply_feature_engineering(test_df, is_train=False)

print(f"\n📊 RESUMEN DESPUÉS DEL FEATURE ENGINEERING:")
print(f"  • Train shape: {train_processed.shape}")
print(f"  • Test shape: {test_processed.shape}")
print(f"  • Nuevas features: {len(new_features_list)}")
print(f"  • Features categóricas nuevas: {[f for f in new_features_list if 'Category' in f or 'Level' in f or 'Combo' in f]}")
print(f"  • Features numéricas nuevas: {[f for f in new_features_list if f not in ['Temp_Cat', 'Hum_Cat', 'N_Level', 'K_Level', 'P_Level', 'Soil_Crop_Combo']]}")

🔧 Aplicando feature engineering a TRAIN...
  1️⃣ Creando ratios de nutrientes...
  2️⃣ Creando índices ambientales...
  3️⃣ Creando categorías basadas en distribuciones...
  4️⃣ Creando interacciones estratégicas...
  5️⃣ Features de balance y ratios avanzados...
  ✅ 16 nuevas features creadas
🔧 Aplicando feature engineering a TEST...
  1️⃣ Creando ratios de nutrientes...
  2️⃣ Creando índices ambientales...
  3️⃣ Creando categorías basadas en distribuciones...
  4️⃣ Creando interacciones estratégicas...
  5️⃣ Features de balance y ratios avanzados...
  ✅ 16 nuevas features creadas

📊 RESUMEN DESPUÉS DEL FEATURE ENGINEERING:
  • Train shape: (750000, 25)
  • Test shape: (250000, 24)
  • Nuevas features: 16
  • Features categóricas nuevas: ['N_Level', 'K_Level', 'P_Level', 'Soil_Crop_Combo', 'Dominant_NPK_Level']
  • Features numéricas nuevas: ['N_P_ratio', 'N_K_ratio', 'P_K_ratio', 'Total_NPK', 'Temp_Hum_index', 'Moist_Balance', 'Environ_Stress', 'NPK_Balance', 'Dominant_NPK_Level', 'T

## 🔢 Codificación de Variables Categóricas

### 🎯 Estrategia de Encoding por Modelo:
- **Random Forest**: Puede manejar categóricas directamente, pero usaremos Label Encoding
- **LightGBM**: Nativo con categóricas, pero Label Encoding es más estable
- **XGBoost**: Requiere encoding numérico - usaremos Label Encoding

**Decisión**: Usar **Label Encoding** para todas las categóricas para máxima compatibilidad.

In [4]:
def apply_encoding(train_df, test_df, target_col='Fertilizer Name'):
    """
    Aplica Label Encoding a todas las variables categóricas
    Mantiene consistencia entre train y test
    Maneja correctamente el caso donde test no tiene columna objetivo (competición Kaggle)
    """
    print("🔢 Aplicando Label Encoding...\n")
    
    # Identificar variables categóricas
    categorical_cols = [
        'Soil Type', 'Crop Type', 'Temp_Cat', 'Hum_Cat', 
        'N_Level', 'K_Level', 'P_Level', 'Soil_Crop_Combo'
    ]
    
    # Verificar que existan en ambos datasets (excluyendo target)
    existing_categorical = [col for col in categorical_cols if col in train_df.columns and col in test_df.columns]
    
    print(f"Variables categóricas a codificar: {existing_categorical}")
    
    # Copiar datasets
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    # Diccionario para almacenar encoders
    label_encoders = {}
    
    # Aplicar Label Encoding a features (no target)
    for col in existing_categorical:
        print(f"  • {col}:")
        
        # Crear encoder
        le = LabelEncoder()
        
        # Combinar valores únicos de train y test para consistencia
        combined_values = pd.concat([train_df[col], test_df[col]]).astype(str)
        le.fit(combined_values)
        
        # Aplicar encoding
        train_encoded[f'{col}_encoded'] = le.transform(train_df[col].astype(str))
        test_encoded[f'{col}_encoded'] = le.transform(test_df[col].astype(str))
        
        # Guardar encoder
        label_encoders[col] = le
        
        # Mostrar mapeo (primeros 5)
        unique_values = train_df[col].unique()[:5]
        print(f"    Total categorías: {len(le.classes_)}")
    
    # Codificar variable objetivo (solo para train - no existe en test)
    if target_col in train_encoded.columns:
        print(f"\n🎯 Codificando variable objetivo: {target_col}")
        target_encoder = LabelEncoder()
        train_encoded[f'{target_col}_encoded'] = target_encoder.fit_transform(train_encoded[target_col])
        label_encoders[target_col] = target_encoder
        print(f"    Total clases objetivo: {len(target_encoder.classes_)}")
        print(f"    Ejemplos: {dict(list(zip(target_encoder.classes_[:5], range(5))))}")
        print(f"    ✅ Objetivo solo en train (correcto para competición Kaggle)")
    else:
        print(f"\n⚠️  Variable objetivo '{target_col}' no encontrada en train")
    
    return train_encoded, test_encoded, label_encoders

# Aplicar encoding
train_encoded, test_encoded, encoders_dict = apply_encoding(train_processed, test_processed)

print(f"\n📊 RESUMEN DESPUÉS DEL ENCODING:")
print(f"  • Train shape: {train_encoded.shape}")
print(f"  • Test shape: {test_encoded.shape}")
print(f"  • Encoders creados: {len(encoders_dict)}")
print(f"  • Variables codificadas: {list(encoders_dict.keys())}")

# Identificar todas las columnas numéricas para escalado
numeric_cols_original = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
numeric_cols_new = [col for col in new_features_list if col not in ['Temp_Cat', 'Hum_Cat', 'N_Level', 'K_Level', 'P_Level', 'Soil_Crop_Combo']]
encoded_cols = [col for col in train_encoded.columns if col.endswith('_encoded')]

all_numeric_cols = numeric_cols_original + numeric_cols_new + encoded_cols

print(f"\n📈 COLUMNAS PARA ESCALADO:")
print(f"  • Originales: {numeric_cols_original}")
print(f"  • Nuevas numéricas: {numeric_cols_new}")
print(f"  • Codificadas: {encoded_cols}")
print(f"  • Total para escalado: {len(all_numeric_cols)}")

🔢 Aplicando Label Encoding...

Variables categóricas a codificar: ['Soil Type', 'Crop Type', 'Temp_Cat', 'Hum_Cat', 'N_Level', 'K_Level', 'P_Level', 'Soil_Crop_Combo']
  • Soil Type:
    Total categorías: 5
  • Crop Type:
    Total categorías: 11
  • Temp_Cat:
    Total categorías: 4
  • Hum_Cat:
    Total categorías: 3
  • N_Level:
    Total categorías: 3
  • K_Level:
    Total categorías: 3
  • P_Level:
    Total categorías: 3
  • Soil_Crop_Combo:
    Total categorías: 55

🎯 Codificando variable objetivo: Fertilizer Name
    Total clases objetivo: 7
    Ejemplos: {'10-26-26': 0, '14-35-14': 1, '17-17-17': 2, '20-20': 3, '28-28': 4}
    ✅ Objetivo solo en train (correcto para competición Kaggle)

📊 RESUMEN DESPUÉS DEL ENCODING:
  • Train shape: (750000, 34)
  • Test shape: (250000, 32)
  • Encoders creados: 9
  • Variables codificadas: ['Soil Type', 'Crop Type', 'Temp_Cat', 'Hum_Cat', 'N_Level', 'K_Level', 'P_Level', 'Soil_Crop_Combo', 'Fertilizer Name']

📈 COLUMNAS PARA ESCALADO:
  •

## 📏 Escalado de Datos

### 🎯 Estrategia de Escalado:
- **StandardScaler**: Para variables con distribuciones normales
- **Solo features comunes**: Train y test deben tener las mismas columnas
- **Excluir variable objetivo**: Solo existe en train (competición Kaggle)
- **Consistencia train/test**: Ajustar scaler solo en train, aplicar a test

> **Nota**: Aunque Random Forest y XGBoost no requieren escalado, lo aplicamos para:
> - Facilitar comparaciones entre modelos
> - Mejorar convergencia de algunos algoritmos
> - Mantener consistencia en el pipeline

In [5]:
# 🔍 DIAGNÓSTICO: Verificar columnas antes del escalado
print("🔍 DIAGNÓSTICO PRE-ESCALADO:\n")

print(f"📊 COLUMNAS EN TRAIN ({len(train_encoded.columns)}):")
print(f"  Primeras 10: {list(train_encoded.columns[:10])}")
print(f"  Últimas 10: {list(train_encoded.columns[-10:])}")

print(f"\n📊 COLUMNAS EN TEST ({len(test_encoded.columns)}):")
print(f"  Primeras 10: {list(test_encoded.columns[:10])}")
print(f"  Últimas 10: {list(test_encoded.columns[-10:])}")

# Verificar diferencias críticas
train_only = set(train_encoded.columns) - set(test_encoded.columns)
test_only = set(test_encoded.columns) - set(train_encoded.columns)
common_cols = set(train_encoded.columns) & set(test_encoded.columns)

print(f"\n🔍 ANÁLISIS DE COLUMNAS:")
print(f"  • Solo en train: {len(train_only)} - {list(train_only)}")
print(f"  • Solo en test: {len(test_only)} - {list(test_only)}")
print(f"  • Comunes: {len(common_cols)}")

# Verificar columnas para escalado
print(f"\n📈 VERIFICACIÓN DE COLUMNAS PARA ESCALADO:")
print(f"  • all_numeric_cols definidas: {len(all_numeric_cols)}")
print(f"  • Muestra: {all_numeric_cols[:10]}")

# Verificar cuáles existen realmente
existing_in_train = [col for col in all_numeric_cols if col in train_encoded.columns]
existing_in_test = [col for col in all_numeric_cols if col in test_encoded.columns]
existing_in_both = [col for col in all_numeric_cols if col in train_encoded.columns and col in test_encoded.columns]

print(f"  • Existen en train: {len(existing_in_train)}")
print(f"  • Existen en test: {len(existing_in_test)}")
print(f"  • Existen en ambos: {len(existing_in_both)}")

if 'Fertilizer Name_encoded' in existing_in_both:
    print(f"  ⚠️  Variable objetivo en lista de features para escalado")
else:
    print(f"  ✅ Variable objetivo NO en lista de features comunes (correcto)")

print(f"\n✅ Diagnóstico completado - procediendo con escalado...")

🔍 DIAGNÓSTICO PRE-ESCALADO:

📊 COLUMNAS EN TRAIN (34):
  Primeras 10: ['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous', 'Fertilizer Name', 'N_P_ratio']
  Últimas 10: ['Temp_Moist_inter', 'Soil Type_encoded', 'Crop Type_encoded', 'Temp_Cat_encoded', 'Hum_Cat_encoded', 'N_Level_encoded', 'K_Level_encoded', 'P_Level_encoded', 'Soil_Crop_Combo_encoded', 'Fertilizer Name_encoded']

📊 COLUMNAS EN TEST (32):
  Primeras 10: ['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous', 'N_P_ratio', 'N_K_ratio']
  Últimas 10: ['Dominant_NPK_Level', 'Temp_Moist_inter', 'Soil Type_encoded', 'Crop Type_encoded', 'Temp_Cat_encoded', 'Hum_Cat_encoded', 'N_Level_encoded', 'K_Level_encoded', 'P_Level_encoded', 'Soil_Crop_Combo_encoded']

🔍 ANÁLISIS DE COLUMNAS:
  • Solo en train: 2 - ['Fertilizer Name_encoded', 'Fertilizer Name']
  • Solo en test: 0 - []
  • Comunes: 32

📈 VERIFICACIÓN DE COLUMNAS PARA E

In [6]:
def apply_scaling(train_df, test_df, numeric_columns, target_col='Fertilizer Name_encoded'):
    """
    Aplica StandardScaler a las columnas numéricas
    Mantiene separados los features de la variable objetivo
    Maneja correctamente el caso donde test no tiene columna objetivo (competición Kaggle)
    """
    print("📏 Aplicando escalado con StandardScaler...\n")
    
    # Copiar datasets
    train_scaled = train_df.copy()
    test_scaled = test_df.copy()
    
    # Filtrar columnas que realmente existen en AMBOS datasets
    # Excluir columna objetivo de las features a escalar
    existing_numeric_train = [col for col in numeric_columns if col in train_df.columns and col != target_col]
    existing_numeric_test = [col for col in numeric_columns if col in test_df.columns]
    
    # Solo escalar columnas que existen en ambos datasets
    existing_numeric = [col for col in existing_numeric_train if col in existing_numeric_test]
    
    print(f"Columnas a escalar: {len(existing_numeric)}")
    print(f"Muestra: {existing_numeric[:10]}...")
    
    if target_col in train_df.columns and target_col not in test_df.columns:
        print(f"✅ Correcto: Variable objetivo '{target_col}' solo en train (competición Kaggle)")
    
    # Crear y ajustar scaler
    scaler = StandardScaler()
    
    # Ajustar solo en train con las features comunes
    train_features = train_df[existing_numeric]
    scaler.fit(train_features)
    
    # Aplicar escalado
    train_scaled_features = scaler.transform(train_features)
    test_scaled_features = scaler.transform(test_df[existing_numeric])
    
    # Reemplazar en los DataFrames
    train_scaled[existing_numeric] = train_scaled_features
    test_scaled[existing_numeric] = test_scaled_features
    
    print(f"✅ Escalado aplicado exitosamente")
    print(f"\n📊 ESTADÍSTICAS POST-ESCALADO:")
    print(f"  • Media de features escaladas (train): {train_scaled[existing_numeric].mean().mean():.3f}")
    print(f"  • Std de features escaladas (train): {train_scaled[existing_numeric].std().mean():.3f}")
    print(f"  • Rango medio escalado: [{train_scaled[existing_numeric].min().min():.2f}, {train_scaled[existing_numeric].max().max():.2f}]")
    
    return train_scaled, test_scaled, scaler, existing_numeric

# Aplicar escalado
train_final, test_final, scaler_fitted, numeric_cols_used = apply_scaling(
    train_encoded, test_encoded, all_numeric_cols
)

print(f"\n🎯 DATASETS FINALES PREPARADOS:")
print(f"  • Train shape: {train_final.shape}")
print(f"  • Test shape: {test_final.shape}")
print(f"  • Features numéricas escaladas: {len(numeric_cols_used)}")
print(f"  • Variable objetivo disponible: {'Fertilizer Name_encoded' in train_final.columns}")

# Verificar que no hay valores nulos después del procesamiento
print(f"\n🔍 VERIFICACIÓN DE CALIDAD:")
print(f"  • Nulos en train: {train_final.isnull().sum().sum()}")
print(f"  • Nulos en test: {test_final.isnull().sum().sum()}")
print(f"  • Infinitos en train: {np.isinf(train_final.select_dtypes(include=[np.number])).sum().sum()}")
print(f"  • Infinitos en test: {np.isinf(test_final.select_dtypes(include=[np.number])).sum().sum()}")

if train_final.isnull().sum().sum() == 0 and test_final.isnull().sum().sum() == 0:
    print("  ✅ Datasets limpios - sin valores nulos o infinitos")
else:
    print("  ⚠️ Revisar valores nulos o infinitos")

📏 Aplicando escalado con StandardScaler...

Columnas a escalar: 24
Muestra: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'N_P_ratio', 'N_K_ratio', 'P_K_ratio', 'Total_NPK']...
✅ Correcto: Variable objetivo 'Fertilizer Name_encoded' solo en train (competición Kaggle)
✅ Escalado aplicado exitosamente

📊 ESTADÍSTICAS POST-ESCALADO:
  • Media de features escaladas (train): -0.000
  • Std de features escaladas (train): 1.000
  • Rango medio escalado: [-2.82, 8.92]

🎯 DATASETS FINALES PREPARADOS:
  • Train shape: (750000, 34)
  • Test shape: (250000, 32)
  • Features numéricas escaladas: 24
  • Variable objetivo disponible: True

🔍 VERIFICACIÓN DE CALIDAD:
  • Nulos en train: 0
  • Nulos en test: 0
  • Infinitos en train: 0
  • Infinitos en test: 0
  ✅ Datasets limpios - sin valores nulos o infinitos


## 🔄 División en Conjuntos de Entrenamiento y Validación

### 🎯 Configuración de División:
- **Proporción**: 80% entrenamiento, 20% validación
- **Random State**: 513 (para reproducibilidad)
- **Estratificada**: Por tipo de fertilizante (variable objetivo)
- **Separación**: X (features) e y (target) claramente definidos

In [7]:
def create_train_validation_split(df, target_col='Fertilizer Name_encoded', test_size=0.2, random_state=513):
    """
    Crea división estratificada de entrenamiento y validación
    """
    print(f"🔄 Creando división train/validation...\n")
    
    # Definir features y target
    feature_cols = [col for col in df.columns if col not in [target_col, 'Fertilizer Name']]
    
    X = df[feature_cols]
    y = df[target_col]
    
    print(f"📊 CONFIGURACIÓN DE DIVISIÓN:")
    print(f"  • Total muestras: {len(df):,}")
    print(f"  • Features: {len(feature_cols)}")
    print(f"  • Clases objetivo: {y.nunique()}")
    print(f"  • Proporción validación: {test_size*100}%")
    print(f"  • Random state: {random_state}")
    
    # División estratificada
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, 
        test_size=test_size, 
        random_state=random_state, 
        stratify=y
    )
    
    print(f"\n✅ DIVISIÓN COMPLETADA:")
    print(f"  • Train: {len(X_train):,} muestras ({len(X_train)/len(df)*100:.1f}%)")
    print(f"  • Validation: {len(X_val):,} muestras ({len(X_val)/len(df)*100:.1f}%)")
    
    # Verificar distribución estratificada
    train_dist = y_train.value_counts(normalize=True).sort_index()
    val_dist = y_val.value_counts(normalize=True).sort_index()
    
    print(f"\n📊 VERIFICACIÓN ESTRATIFICACIÓN:")
    print(f"  • Diferencia máxima en distribución: {abs(train_dist - val_dist).max():.4f}")
    if abs(train_dist - val_dist).max() < 0.01:
        print(f"  ✅ Estratificación exitosa (diferencia < 1%)")
    else:
        print(f"  ⚠️ Verificar estratificación")
    
    return X_train, X_val, y_train, y_val, feature_cols

# Crear división
X_train, X_val, y_train, y_val, feature_columns = create_train_validation_split(train_final)

print(f"\n🎯 DATASETS FINALES PARA MODELADO:")
print(f"  • X_train: {X_train.shape}")
print(f"  • X_val: {X_val.shape}")
print(f"  • y_train: {y_train.shape} (clases únicas: {y_train.nunique()})")
print(f"  • y_val: {y_val.shape} (clases únicas: {y_val.nunique()})")
print(f"  • Test completo: {test_final[feature_columns].shape}")

# Preparar dataset de test para predicción
X_test = test_final[feature_columns]

print(f"\n📋 RESUMEN DE FEATURES:")
print(f"  • Total features: {len(feature_columns)}")
print(f"  • Features originales: {len(numeric_cols_original)}")
print(f"  • Features nuevas: {len([f for f in feature_columns if f in new_features_list or f.endswith('_encoded')])}")
print(f"  • Todas escaladas: ✅")

🔄 Creando división train/validation...

📊 CONFIGURACIÓN DE DIVISIÓN:
  • Total muestras: 750,000
  • Features: 32
  • Clases objetivo: 7
  • Proporción validación: 20.0%
  • Random state: 513

✅ DIVISIÓN COMPLETADA:
  • Train: 600,000 muestras (80.0%)
  • Validation: 150,000 muestras (20.0%)

📊 VERIFICACIÓN ESTRATIFICACIÓN:
  • Diferencia máxima en distribución: 0.0000
  ✅ Estratificación exitosa (diferencia < 1%)

🎯 DATASETS FINALES PARA MODELADO:
  • X_train: (600000, 32)
  • X_val: (150000, 32)
  • y_train: (600000,) (clases únicas: 7)
  • y_val: (150000,) (clases únicas: 7)
  • Test completo: (250000, 32)

📋 RESUMEN DE FEATURES:
  • Total features: 32
  • Features originales: 6
  • Features nuevas: 24
  • Todas escaladas: ✅


## 💾 Guardar Datasets Procesados

### 🎯 Estructura de Archivos:
- **Datos principales**: X_train, X_val, y_train, y_val, X_test (formato .parquet)
- **Metadatos**: encoders, scaler, mapeos de columnas (.pkl)
- **Datasets completos**: train/test procesados con índice 'id' (.parquet)

### 📦 Ventajas del formato .parquet:
- **Eficiencia**: Archivos 60-80% más pequeños que CSV
- **Velocidad**: Carga/escritura más rápida
- **Tipos preservados**: Mantiene automáticamente dtypes e índices
- **Compatibilidad**: Funciona con pandas, sklearn y todos los frameworks ML

### 📁 Ubicación: `/data/processed/`

In [8]:
def save_processed_datasets():
    """
    Guarda todos los datasets procesados y metadatos necesarios
    OPTIMIZADO: Usando formato .parquet (más eficiente, mantiene índices y tipos automáticamente)
    No crea archivos redundantes ya que la información de ID está en el índice del dataset
    """
    print("💾 Guardando datasets procesados (formato .parquet optimizado)...\n")
    
    # 1. DATASETS PRINCIPALES
    print("1️⃣ Guardando datasets de entrenamiento y validación...")
    
    # Datasets de entrenamiento (DataFrames)
    X_train.to_parquet(f'{processed_dir}/X_train.parquet')
    X_val.to_parquet(f'{processed_dir}/X_val.parquet')
    
    # Variables objetivo (Series) - convertir a DataFrame para guardar en parquet
    y_train.to_frame().to_parquet(f'{processed_dir}/y_train.parquet')
    y_val.to_frame().to_parquet(f'{processed_dir}/y_val.parquet')
    
    # Dataset de test
    X_test.to_parquet(f'{processed_dir}/X_test.parquet')
    
    print(f"  ✅ X_train.parquet: {X_train.shape}")
    print(f"  ✅ X_val.parquet: {X_val.shape}")
    print(f"  ✅ y_train.parquet: {y_train.shape}")
    print(f"  ✅ y_val.parquet: {y_val.shape}")
    print(f"  ✅ X_test.parquet: {X_test.shape}")
    
    # 2. METADATOS Y CONFIGURACIONES
    print("\n2️⃣ Guardando metadatos y configuraciones...")
    
    # Guardar encoders y scaler
    joblib.dump(encoders_dict, f'{processed_dir}/label_encoders.pkl')
    joblib.dump(scaler_fitted, f'{processed_dir}/standard_scaler.pkl')
    print(f"  ✅ label_encoders.pkl: {len(encoders_dict)} encoders")
    print(f"  ✅ standard_scaler.pkl: scaler para {len(numeric_cols_used)} features")
    
    # Guardar lista de columnas
    feature_info = {
        'feature_columns': feature_columns,
        'numeric_original': numeric_cols_original,
        'numeric_new': numeric_cols_new,
        'categorical_original': ['Soil Type', 'Crop Type'],
        'categorical_new': [f for f in new_features_list if f in ['Temp_Cat', 'Hum_Cat', 'N_Level', 'K_Level', 'P_Level', 'Soil_Crop_Combo']],
        'encoded_columns': [col for col in encoded_cols if col in feature_columns],
        'target_column': 'Fertilizer Name_encoded',
        'target_original': 'Fertilizer Name'
    }
    
    joblib.dump(feature_info, f'{processed_dir}/feature_info.pkl')
    print(f"  ✅ feature_info.pkl: información de {len(feature_columns)} features")
    
    # 3. DATASETS COMPLETOS CON ÍNDICE ID (OPTIMIZADO)
    print("\n3️⃣ Guardando datasets completos con índice 'id'...")
    
    # Train completo procesado (mantiene el índice 'id' automáticamente con parquet)
    train_complete = train_final.copy()
    train_complete.to_parquet(f'{processed_dir}/train_complete_processed.parquet')
    print(f"  ✅ train_complete_processed.parquet: {train_complete.shape} (con índice 'id')")
    
    # Test completo procesado (mantiene el índice 'id' automáticamente con parquet)
    test_complete = test_final.copy()
    test_complete.to_parquet(f'{processed_dir}/test_complete_processed.parquet')
    print(f"  ✅ test_complete_processed.parquet: {test_complete.shape} (con índice 'id')")
    
    print("\n🎉 Todos los datasets guardados exitosamente")
    return True

# Ejecutar guardado
save_success = save_processed_datasets()

if save_success:
    print(f"\n🎉 ¡PREPROCESAMIENTO COMPLETADO EXITOSAMENTE!\n")
    
    print(f"📁 ARCHIVOS GUARDADOS EN: {processed_dir}")
    print(f"\n📊 RESUMEN FINAL:")
    print(f"  • Datasets listos para: Random Forest, LightGBM, XGBoost")
    print(f"  • Features totales: {len(feature_columns)}")
    print(f"  • Train/Validation: 80/20 estratificado")
    print(f"  • Random state: 513")
    print(f"  • Escalado: StandardScaler aplicado")
    print(f"  • Encoding: Label Encoding para categóricas")
    
    print(f"\n🚀 PRÓXIMOS PASOS:")
    print(f"  1. Cargar datasets desde /data/processed/")
    print(f"  2. Implementar modelos TIER 1")
    print(f"  3. Evaluar performance con validation set")
    print(f"  4. Generar predicciones para submission")
    
    print(f"\n✅ ¡Listos para modelado!")
else:
    print(f"❌ Error en el guardado")

💾 Guardando datasets procesados (formato .parquet optimizado)...

1️⃣ Guardando datasets de entrenamiento y validación...
  ✅ X_train.parquet: (600000, 32)
  ✅ X_val.parquet: (150000, 32)
  ✅ y_train.parquet: (600000,)
  ✅ y_val.parquet: (150000,)
  ✅ X_test.parquet: (250000, 32)

2️⃣ Guardando metadatos y configuraciones...
  ✅ label_encoders.pkl: 9 encoders
  ✅ standard_scaler.pkl: scaler para 24 features
  ✅ feature_info.pkl: información de 32 features

3️⃣ Guardando datasets completos con índice 'id'...
  ✅ train_complete_processed.parquet: (750000, 34) (con índice 'id')
  ✅ test_complete_processed.parquet: (250000, 32) (con índice 'id')

🎉 Todos los datasets guardados exitosamente

🎉 ¡PREPROCESAMIENTO COMPLETADO EXITOSAMENTE!

📁 ARCHIVOS GUARDADOS EN: ../data/processed

📊 RESUMEN FINAL:
  • Datasets listos para: Random Forest, LightGBM, XGBoost
  • Features totales: 32
  • Train/Validation: 80/20 estratificado
  • Random state: 513
  • Escalado: StandardScaler aplicado
  • Encodin

## ✅ Verificación Final y Próximos Pasos

### 🔍 Resumen del Preprocesamiento:

1. **✅ Feature Engineering Completado**: Todas las variables del EDA recreadas
2. **✅ Encoding Aplicado**: Label Encoding para máxima compatibilidad 
3. **✅ Escalado Realizado**: StandardScaler para normalización
4. **✅ División Train/Val**: 80/20 estratificado con random_state=513
5. **✅ Datasets Guardados**: Listos para modelos en `/data/processed/`

### 🎯 Modelos Preparados Para:
- **Random Forest**: Baseline robusto
- **LightGBM**: Gradient boosting eficiente  
- **XGBoost**: Gradient boosting de alta performance

### 📁 Archivos Generados (Optimizados):
```
data/processed/
├── X_train.parquet, X_val.parquet, y_train.parquet, y_val.parquet  # Datasets divididos
├── X_test.parquet                                                    # Test para predicción
├── label_encoders.pkl                                                # Encoders categóricos
├── standard_scaler.pkl                                               # Scaler numérico
├── feature_info.pkl                                                  # Metadatos de features
└── train/test_complete_processed.parquet                            # Datasets completos (con índice 'id')
```

### 📄 Cómo cargar los archivos:
```python
# Cargar datasets principales
X_train = pd.read_parquet('../data/processed/X_train.parquet')
y_train = pd.read_parquet('../data/processed/y_train.parquet')

# Cargar datasets completos (con índice 'id')
test_complete = pd.read_parquet('../data/processed/test_complete_processed.parquet')
```

### 🚀 ¡Preprocesamiento completado! Listos para fase de modelado.